# Mean-Variance Portfolio Demo
Data source: `Simulasi Portofolio Tambang.xlsx` (sheet `HBA HMA`).
This notebook shows target-return and target-volatility optimization for a chosen asset set.


In [1]:
import argparse
from datetime import datetime, date
import numpy as np
import pandas as pd
from scipy.optimize import minimize

MONTH_MAP = {
    'januari': 1, 'februari': 2, 'maret': 3, 'april': 4, 'mei': 5, 'juni': 6,
    'juli': 7, 'agustus': 8, 'september': 9, 'oktober': 10, 'november': 11, 'desember': 12,
}

def parse_date(x):
    if pd.isna(x):
        return None
    if isinstance(x, datetime):
        return x.date()
    s = str(x).strip()
    if not s:
        return None
    try:
        return pd.to_datetime(s).date()
    except Exception:
        pass
    parts = s.lower().split()
    if len(parts) == 2 and parts[0] in MONTH_MAP:
        m = MONTH_MAP[parts[0]]
        y = int(parts[1])
        return date(y, m, 1)
    return None

def load_prices(path, sheet_name):
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None)
    header = raw.loc[1].tolist()
    cols = []
    for idx, val in enumerate(header):
        if idx == 0:
            cols.append('Komoditas')
        elif idx == 1:
            cols.append('Deskripsi')
        else:
            cols.append(parse_date(val))
    rows = raw.loc[2:14].copy()
    rows.columns = cols
    rows = rows.dropna(axis=1, how='all')
    date_cols = [c for c in rows.columns if isinstance(c, date)]
    for c in date_cols:
        rows[c] = pd.to_numeric(rows[c], errors='coerce')
    rows = rows.set_index('Komoditas')
    return rows[date_cols]

def compute_log_returns(price_df):
    return np.log(price_df / price_df.shift(1)).dropna(how='any')

def annualize_mean_cov(returns_df, periods_per_year=12):
    mu = returns_df.mean() * periods_per_year
    cov = returns_df.cov() * periods_per_year
    return mu.values, cov.values, mu

def optimize_target_return(mu, cov, target_return, bounds):
    n = len(mu)
    def port_var(w):
        return w.T @ cov @ w
    cons = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w: w @ mu - target_return},
    )
    w0 = np.ones(n) / n
    return minimize(port_var, w0, bounds=bounds, constraints=cons)

def optimize_target_vol(mu, cov, target_vol, bounds):
    n = len(mu)
    def port_vol(w):
        return np.sqrt(w.T @ cov @ w)
    def objective(w):
        return (port_vol(w) - target_vol) ** 2
    cons = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    )
    w0 = np.ones(n) / n
    return minimize(objective, w0, bounds=bounds, constraints=cons)

path = 'Simulasi Portofolio Tambang.xlsx'
sheet = 'HBA HMA'
assets = ['Emas', 'Perak', 'Besi', 'Tembaga']
prices = load_prices(path, sheet)
prices = prices.loc[assets].T
returns = compute_log_returns(prices)
mu, cov, mu_series = annualize_mean_cov(returns)
target_return = float(mu.mean())
target_vol = 0.10
bounds = [(0.05, 0.50)] * len(assets)

print('Date range:', returns.index.min(), 'to', returns.index.max())
print(mu_series)

res_ret = optimize_target_return(mu, cov, target_return, bounds)
w_ret = res_ret.x
exp_ret = w_ret @ mu
vol_ret = np.sqrt(w_ret.T @ cov @ w_ret)

print("\nTarget return:", target_return)
print("Weights:", dict(zip(assets, np.round(w_ret, 4))))
print("Expected return:", exp_ret)
print("Volatility:", vol_ret)

res_vol = optimize_target_vol(mu, cov, target_vol, bounds)
w_vol = res_vol.x
exp_ret2 = w_vol @ mu
vol2 = np.sqrt(w_vol.T @ cov @ w_vol)

print("\nTarget volatility:", target_vol)
print("Weights:", dict(zip(assets, np.round(w_vol, 4))))
print("Expected return:", exp_ret2)
print("Volatility:", vol2)


Date range: 2017-12-01 to 2025-02-01
Annualized mean returns:
  Emas: 9.98%
  Perak: 7.80%
  Besi: 6.79%
  Tembaga: 4.08%

Target return optimization
Target return: 7.16%
  Emas: 46.82%
  Perak: 5.00%
  Besi: 5.00%
  Tembaga: 43.18%
  Expected return: 7.16%
  Volatility: 11.29%

Target volatility optimization
Target volatility: 10.00%
  Emas: 50.00%
  Perak: 5.00%
  Besi: 5.00%
  Tembaga: 40.00%
  Expected return: 7.35%
  Volatility: 11.06%
